# Dieses Notebook war nur zum ausprobieren!!

# Early-Stopping für Softmax-Regression
Implementieren sie das Batch-Gradientenverfahren mit Early Stopping für die Softmax-Regression (ohne Scikit-Learn)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.datasets import load_iris

iris = load_iris()

In [3]:
iris.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

In [4]:
# [classes]
y = iris['target']
print(y.shape)
y[:3]

(150,)


array([0, 0, 0])

In [5]:
# [['petal length', 'petal width']]
X = iris['data'][:, (2,3)]
print(X.shape)
X[:3]

(150, 2)


array([[1.4, 0.2],
       [1.4, 0.2],
       [1.3, 0.2]])

In [6]:
eta = 0.1
n_iter = 1000
m = len(X)

**Softmax-Score für Kategorie k**  
$$s_k(x) = (\theta^{(k)})^T*x$$

In [7]:
theta = np.random.randn(3, 2)
print(theta.shape)
theta

(3, 2)


array([[ 0.38078309, -2.54293602],
       [ 1.34484207, -0.31075688],
       [-0.13690439,  0.6465947 ]])

In [8]:
X.shape

(150, 2)

In [9]:
#softmax-score für jeden datenpunkt und jede kategorie
# [
# [cat1, cat2], -> erster Datenpunkt
# [cat1, cat2] -> zweiter Datenpunkt
#]
score = []
for row in X:
    row_score = []
    for cat in theta:
        cat_score = cat.T.dot(row)
        row_score.append(cat_score)
    score.append(row_score)
score = np.array(score)
print(score.shape)
score[:2]

(150, 3)


array([[ 0.02450913,  1.82062752, -0.06234721],
       [ 0.02450913,  1.82062752, -0.06234721]])

**Softmax-Funktion**  
$$\hat p_k = \sigma(s(x())_k = \frac{exp(s_k(x))}{\sum_{j=1}^Kexp(s_j(x))}$$
* $K$ ist die Anzahl der Kategorien
* $s(x)$ ist ein Vektor der die Scores jeder Kategorie für den Datenpunkt x enthält
* $\sigma(s(x))_k$ ist die anhand der Sores geschätze Wahrscheinlichkeit dafür, dass der Datenpunkt $x$ zur Kategorie $k$ gehört.

In [10]:
softmax = []
for p in score:
    p_softmax = []
    cat_score_sum = np.sum([np.exp(cat_score) for cat_score in p])
    for cat_score in p:
        cat_softmax = np.exp(cat_score) / cat_score_sum
        p_softmax.append(cat_softmax)
    softmax.append(p_softmax)
softmax = np.array(softmax)
print(softmax.shape)
softmax[:2]

(150, 3)


array([[0.12589671, 0.75868008, 0.11542321],
       [0.12589671, 0.75868008, 0.11542321]])

**Vorhersage eines Klassifikators mit Softmax-Regression**  
$$\hat y = argmax \sigma(s(x))_k = argmax s_k(x) = argmax ((\theta^{(k)})^T * x))$$
* Der Operator argmax liefert den Wert einer Variable, der eine Funktion maximiert. Hier liefert er denjenigen Wert für $k$, für den die geschätzte Wahrscheinlichkeit $\sigma(s(x))_k$ maximal wird

In [11]:
probas = []
for proba in softmax:
    probas.append(np.argmax(proba))
probas = np.array(probas)
print(probas.shape)
probas[:2]

(150,)


array([1, 1])

**Kreuzentropie als Kostenfunktion**  
$$J(\Theta) = -\frac{1}{m}\sum_{i=1}^m\sum_{k=1}y_k^{(i)}log(\hat p_k^{(i)})$$
* $y_k^{(i)}$ ist gleich 1, wenn die Zielkategorie des $i^{ten}$ Datenpunkts $k$ ist, sonst 0.

**Gradientenvektor der Kreuzentropie für Kategorie k**  
$$\nabla_{\theta}J(\Theta) = \frac{1}{m}\sum_{i=1}^m(\hat p_k^{(i)} - y_k^{(i)})x^{(i)}$$

In [12]:
print(theta.shape, probas.shape, softmax.shape, score.shape, X.shape, y.shape, sep='\n')

(3, 2)
(150,)
(150, 3)
(150, 3)
(150, 2)
(150,)


In [13]:
one = y == 0
two = y == 1
three = y == 2
y_new = np.array([one,two,three]).astype(np.int).T
print(y_new.shape)
y_new[:3]

(150, 3)


array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]])

In [14]:
# Training
for epoch in range(1000):
    for i, cat_theta in enumerate(theta):
        bla = 1 / m * np.sum((softmax[:,i] - y_new[:,i]).dot(X))
        theta[i] = cat_theta - eta * bla

In [15]:
theta

array([[  50.84406186,   47.92034275],
       [-293.44557302, -295.10117197],
       [ 244.19023193,  244.97373102]])

# Hier beginnt das richtige!!

In [16]:
# Softmax-Score für jede Kategorie und jeden Datenpunkt
def get_score(theta, X):
    score = []
    for cat in theta:
        cat_score = []
        for row in X:
            row_score = cat.T.dot(row)
            cat_score.append(row_score)
        score.append(cat_score)
    score = np.array(score).T
    print(score.shape)
    return score

In [17]:
get_score(theta, X)[:3]

(150, 3)


array([[  80.76575515, -469.84403662,  390.86107091],
       [  80.76575515, -469.84403662,  390.86107091],
       [  75.68134897, -440.49947932,  366.44204772]])

In [18]:
# Softmax-Funktion
def get_softmax(score):
    softmax = []
    for p in score:
        p_softmax = []
        cat_score_sum = np.sum([np.exp(cat_score) for cat_score in p])
        for cat_score in p:
            cat_softmax = np.exp(cat_score) / cat_score_sum
            p_softmax.append(cat_softmax)
        softmax.append(p_softmax)
    softmax = np.array(softmax)
    print(softmax.shape)
    return softmax

In [19]:
get_softmax(get_score(theta, X))[:3]

(150, 3)
(150, 3)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


array([[2.12478750e-135, 0.00000000e+000, 1.00000000e+000],
       [2.12478750e-135, 0.00000000e+000, 1.00000000e+000],
       [5.29947659e-127, 0.00000000e+000, 1.00000000e+000]])

In [20]:
# Prediction
def predict(softmax):
    probas = []
    for proba in softmax:
        probas.append(np.argmax(proba))
    probas = np.array(probas)
    print(probas.shape)
    return probas

In [21]:
predict(get_softmax(get_score(theta, X)))

(150, 3)
(150, 3)
(150,)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [22]:
# Training
# TODO: softmax muss als parameter übergeben werden!!!
# TODO: predict und sgd funktionieren noch nicht
def fit(softmax, X, y, eta=0.1, n_iter=100):
    theta = np.random.randn(3, 2)
    for epoch in range(n_iter):
        for i, cat_theta in enumerate(theta):
            gradient = 1 / m * np.sum((softmax[:,i] - y[:,i]).dot(X))
            theta[i] = cat_theta - eta * gradient
    return theta

In [31]:
softmax = get_softmax(get_score(theta, X)
                      
fit(softmax, X, y_new).shape

SyntaxError: invalid syntax (<ipython-input-31-1d5dcca2bca2>, line 3)

In [27]:
y_new[:3]

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]])

In [24]:
theta
np.app

array([[  55.89038974,   52.96667063],
       [-322.92461453, -324.58021348],
       [ 268.62294557,  269.40644465]])

Reihenfolge
1. score berechnen
2. softmax funktion berechnen
3. trainieren
4. validieren
5. trainieren
6. validieren
7. ...
8. testen

In [45]:
np.vstack(([[1],[2]],[[2]]))

TypeError: vstack() got an unexpected keyword argument 'axis'

In [50]:
np.append([[1],[2]],[[2]], axis=0)

array([[1],
       [2],
       [2]])